In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/US_Accidents_Dec21_updated.csv')

# Data Cleaning

## Addressing Missing Values and Dropping Irrelevant Columns

In [3]:
df.drop(['Number', 'Zipcode', 'Wind_Chill(F)', 'Airport_Code', 'Nautical_Twilight', 'Astronomical_Twilight',
        'Weather_Timestamp'], axis = 1, inplace = True)

In [4]:
# Checking nulls per weather category
pd.DataFrame(df['Precipitation(in)'].isnull().groupby(by=df['Weather_Condition']).sum().sort_values(ascending=False)).head(50)

,Precipitation(in)
Weather_Condition,
Clear,172786
Overcast,72357
Mostly Cloudy,72029
Partly Cloudy,51271
Fair,45103
Scattered Clouds,44052
Cloudy,8959
Haze,6675
Light Rain,3908


### Filling weather condition categories with many nulls that should have no precipitation with 0

In [5]:
df.loc[(df['Precipitation(in)'].isna()) & (df['Weather_Condition'].str.contains('Clear|Overcast|Mostly Cloudy|Partly Cloudy|Fair|Scattered Clouds', regex=True)), ['Precipitation(in)']] = 0

### Filling other weather condition categories with median

In [6]:
df['Precipitation(in)'].fillna(df.groupby('Weather_Condition')['Precipitation(in)'].transform('median'), inplace=True)

### Dropping all other nulls as they only have nulls per category

In [7]:
df.dropna(subset = 'Precipitation(in)', inplace = True)

In [8]:
df['Wind_Speed(mph)'].isnull().groupby(by=df['Wind_Direction']).sum().sort_values(ascending=False)

Wind_Direction
Calm        76022
North        2907
CALM            0
SE              0
WSW             0
WNW             0
W               0
Variable        0
VAR             0
South           0
SW              0
SSW             0
SSE             0
S               0
NW              0
NNW             0
NNE             0
NE              0
N               0
East            0
ESE             0
ENE             0
E               0
West            0
Name: Wind_Speed(mph), dtype: int64

### Filling Wind_Speed nulls with category Wind_Direction: Calm with 0 and category Wind_Direction: North with 0

In [9]:
df.loc[(df['Wind_Speed(mph)'].isna()) & (df['Wind_Direction'].str.contains('Calm', regex=True)), ['Wind_Speed(mph)']] = 0

In [10]:
df.dropna(subset = 'Wind_Speed(mph)', inplace = True)

In [11]:
# Exploring what columns might be correlated to Humidity to support imputation process
df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']].corr()

,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
Temperature(F),1.000000,-0.366206,0.137220,0.210497,0.086516,-0.004348
Humidity(%),-0.366206,1.000000,0.139746,-0.363129,-0.181569,0.074751
Pressure(in),0.137220,0.139746,1.000000,0.035240,-0.054383,0.006759
Visibility(mi),0.210497,-0.363129,0.035240,1.000000,0.037998,-0.114270
Wind_Speed(mph),0.086516,-0.181569,-0.054383,0.037998,1.000000,0.022316
Precipitation(in),-0.004348,0.074751,0.006759,-0.114270,0.022316,1.000000


In [12]:
# Number of rows with null humidity compared to entire df
13151 / df.shape[0]

0.004762606638967183

### Dropping null humidities as they constitute a very small proportion of data and lack of domain knowledge


In [13]:
df.dropna(subset = 'Humidity(%)', inplace = True)

In [14]:
temp = df.isna().sum().sort_values(ascending=False)
temp = temp[temp.values > 0]
temp

Visibility(mi)       7949
Weather_Condition    6600
Sunrise_Sunset       2256
Civil_Twilight       2256
Pressure(in)         2001
City                  132
Wind_Direction         30
Street                  1
dtype: int64

In [15]:
temp.index

Index(['Visibility(mi)', 'Weather_Condition', 'Sunrise_Sunset',
       'Civil_Twilight', 'Pressure(in)', 'City', 'Wind_Direction', 'Street'],
      dtype='object')

### Dropping null humidities as they constitute a very small proportion of data and lack of domain knowledge

In [16]:
df.dropna(subset = ['Visibility(mi)', 'Weather_Condition', 'Sunrise_Sunset',
       'Civil_Twilight', 'Pressure(in)', 'City', 'Wind_Direction', 'Street'], inplace = True)

### Dropping remaining irrelevant columns

In [17]:
df.drop(['ID', 'End_Time'], axis = 1, inplace = True)

In [18]:
df.head()

,Severity,Start_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,Side,...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight
0,3,2016-02-08 00:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,Outerbelt E,R,...,False,False,False,False,False,False,False,False,Night,Night
1,2,2016-02-08 05:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,I-70 E,R,...,False,False,False,False,False,False,False,False,Night,Night
2,2,2016-02-08 06:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,I-75 S,R,...,False,False,False,False,False,False,False,False,Night,Night
3,2,2016-02-08 06:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,I-77 N,R,...,False,False,False,False,False,False,False,False,Night,Night
4,3,2016-02-08 07:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,I-75 S,R,...,False,False,False,False,False,False,False,False,Day,Day


In [19]:
df.drop(['End_Lat', 'End_Lng', 'Description', 'Wind_Direction', 'Country', 'Civil_Twilight'], axis = 1, inplace = True)

In [20]:
df.drop(['Distance(mi)'], axis = 1, inplace = True)

In [21]:
df.columns

Index(['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Street', 'Side',
       'City', 'County', 'State', 'Timezone', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset'],
      dtype='object')

In [24]:
# Exporting cleaned df to csv
df.to_csv('../data/cleaned_df.csv')